### Import

In [2]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

### GPU

In [3]:
torch.cuda.device_count()

cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')

device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

### INIT

In [4]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = '/Volumes/Macintosh HD/DATASETS/GUITAR-FX/Mono'
excl_folders = ['NoFX_mono', 'NoFX_mono_preprocessed']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=6

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
split = datasplit.DataSplit(dataset, shuffle=True)
# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

In [5]:
len(dataset)

163488

### TRAIN SetNET

In [9]:
# model
setnet = models.SettingsNet().to(device)
# optimizer
optimizer_setnet = optim.Adam(setnet.parameters(), lr=0.001)
# loss function
loss_func_setnet = nn.MSELoss(reduction='mean')

In [10]:
print(setnet)

SettingsNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=6, bias=True)
)


In [11]:
# TRAIN and TEST SetNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

start = time.time()

for epoch in range(50):
    train_loss, train_correct, train_results = trainer.train_settings_net(
        model=setnet,
        optimizer=optimizer_setnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch,
        loss_function=loss_func_setnet, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_settings_net(
        model=setnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler,
        loss_function=loss_func_setnet,
        device=device
    )
    
    test_loss, test_correct, test_results = trainer.test_settings_net(
        model=setnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler,
        loss_function=loss_func_setnet,
        device=device
    )
    
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

stop = time.time()
print(f"Training time: {stop - start}s")

 Epoch: 47	[90000/117711 (76%)]	Total Loss: 4.7531	Avg Loss: 0.0001
Train Epoch: 47	[91000/117711 (77%)]	Total Loss: 4.8039	Avg Loss: 0.0001
Train Epoch: 47	[92000/117711 (78%)]	Total Loss: 4.8611	Avg Loss: 0.0001
Train Epoch: 47	[93000/117711 (79%)]	Total Loss: 4.9059	Avg Loss: 0.0001
Train Epoch: 47	[94000/117711 (80%)]	Total Loss: 4.9630	Avg Loss: 0.0001
Train Epoch: 47	[95000/117711 (81%)]	Total Loss: 5.0051	Avg Loss: 0.0001
Train Epoch: 47	[96000/117711 (81%)]	Total Loss: 5.0475	Avg Loss: 0.0001
Train Epoch: 47	[97000/117711 (82%)]	Total Loss: 5.0935	Avg Loss: 0.0001
Train Epoch: 47	[98000/117711 (83%)]	Total Loss: 5.1481	Avg Loss: 0.0001
Train Epoch: 47	[99000/117711 (84%)]	Total Loss: 5.1909	Avg Loss: 0.0001
Train Epoch: 47	[100000/117711 (85%)]	Total Loss: 5.2437	Avg Loss: 0.0001
Train Epoch: 47	[101000/117711 (86%)]	Total Loss: 5.2925	Avg Loss: 0.0001
Train Epoch: 47	[102000/117711 (87%)]	Total Loss: 5.3421	Avg Loss: 0.0001
Train Epoch: 47	[103000/117711 (87%)]	Total Loss: 5.3

In [12]:
# SAVE
models_folder = 'saved_models'
model_name = '20200921_setnet'
results_folder = 'saved_results'

In [13]:
# SAVE MODEL
torch.save(setnet, '%s/%s' % (models_folder, model_name))

In [15]:
# SAVE RESULTS
all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)
all_train_results_npy = np.array(all_train_results)

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)
all_val_results_npy = np.array(all_val_results)

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)
all_test_results_npy = np.array(all_test_results)

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_losses')), arr=all_train_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_correct')), arr=all_train_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_results')), arr=all_train_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_losses')), arr=all_val_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_correct')), arr=all_val_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_results')), arr=all_val_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_losses')), arr=all_test_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_correct')), arr=all_test_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_results')), arr=all_test_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'fx_labels')), arr=fx_labels_npy)